###Строим запрос к нашему сервису

In [54]:
#!pip uninstall -v scikit-learn
!pip install -v scikit-learn

Using pip 22.0.4 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("/content/X_test.csv")
y_test = pd.read_csv("/content/y_test.csv")

In [56]:
import requests
import urllib.request
import json 

In [ ]:
# Пример данных
data = (X_test)
print(data)

# формируем запрос
def send_json(x):
#     gender, tenure, PhoneService, TotalCharges, StreamingMovies, StreamingTV, TechSupport = x
    tenure, PhoneService, TotalCharges, StreamingMovies, StreamingTV, TechSupport,gender = x
    #print(description, company_profile, benefits)
    body = {
    "tenure":tenure,
    "PhoneService":PhoneService,
    "TotalCharges":TotalCharges,
    "StreamingMovies":StreamingMovies, 
    "StreamingTV":StreamingTV,
    "TechSupport":TechSupport,
    "gender":gender
#     "gender_Female":[gender_Female],
#     "gender_Male":[gender_Male],
    }
    myurl = 'http://127.0.0.1:5000/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

# обращение к серверу с запросом из одного набора (его построили руками выше - data)
if __name__ == '__main__':
    response = send_json(data)
    print('предсказание',response)

      tenure  PhoneService  TotalCharges  StreamingMovies  StreamingTV  \
0          1             0         24.80                0            0   
1         41             1        996.45                0            0   
2         52             1       1031.70                0            0   
3          1             1         76.35                0            0   
4         67             1       3260.10                0            0   
...      ...           ...           ...              ...          ...   
2320      71             1       1901.00                0            0   
2321      29             0       1636.95                1            0   
2322       7             1        631.85                0            1   
2323      32             1       3145.90                1            1   
2324      59             1       5597.65                0            1   

      TechSupport  gender  
0               0  Female  
1               0    Male  
2               0  Female  

ConnectionError: ignored

###Сделаем обработку массы запросов:

1. загрузим данные X_test

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# path = '/content/gdrive/MyDrive/2_ver_actual Артем Зраев. 10.2020/Lection9/'
path2='/home/nal/GB_docker_flask_example/ml_business/project_churn/'

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [ ]:
# response = send_json(X_test[["gender",
#     "tenure",
#     "PhoneService",
#     "TotalCharges",
#     "StreamingMovies", 
#     "StreamingTV",
#     "TechSupport"]].iloc[0,:])

In [ ]:
response = send_json(X_test[["gender",
    "tenure",
    "PhoneService",
    "TotalCharges",
    "StreamingMovies", 
    "StreamingTV",
    "TechSupport"]].iloc[0,:])

In [ ]:
response

In [ ]:
X_test[['description', 'company_profile', 'benefits']].head()

In [58]:
import urllib.request
import json      

def get_prediction(description, company_profile, benefits):
    body = {'description': description, 
                            'company_profile': company_profile,
                            'benefits': benefits} 

    myurl = "http://127.0.0.1:5000/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [ ]:
get_prediction('test', 'test job', '888888888 ')

In [ ]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits']].iloc[:500].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2]), 1)

In [ ]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

In [ ]:
get_prediction("", "", None)

###Давайте подумаем, где наше api может сломаться

3. Сделаем N запросов и оценим время

In [ ]:
N = 50

In [ ]:
%%time
predictions = X_test[['description', 'company_profile', 
                      'benefits']].iloc[:N,:].apply(lambda x: send_json(x),1)

In [ ]:
predictions.values[:5]

###4. Посчитаем метрику roc-auc

In [ ]:
y_test.head()

,Churn
0,1
1,0
2,0
3,1
4,0


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

5. Анализ : в целом похоже

In [ ]:
X_test.head()

,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,gender
0,1,0,24.80,0,0,0,Female
1,41,1,996.45,0,0,0,Male
2,52,1,1031.70,0,0,0,Female
3,1,1,76.35,0,0,0,Female
4,67,1,3260.10,0,0,1,Male


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])